In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pathlib import Path
main_folder = Path('/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230331_sqli_colab')


In [22]:
%cd $main_folder
%ls -lah

/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230323_sqli_colab
total 81K
-rw------- 1 root root  683 Mar 24 12:24 classical_models.py
-rw------- 1 root root  596 Mar 29 13:32 config.ini
drwx------ 2 root root 4.0K Mar 23 13:32 datasets/
-rw------- 1 root root  35K Mar 30 14:22 detect_sqi_with_transformers.ipynb
-rw------- 1 root root 7.6K Mar 24 12:24 ensemble_models.py
-rw------- 1 root root 6.1K Mar 30 13:46 experiments.py
-rw------- 1 root root    3 Jan 16 13:22 LICENSE.md
-rw------- 1 root root 1.1K Mar 29 13:30 main.py
drwx------ 2 root root 4.0K Mar 23 13:45 __pycache__/
-rw------- 1 root root 5.4K Mar 24 11:28 README.md
drwx------ 2 root root 4.0K Mar 30 13:56 results/
-rw------- 1 root root 5.1K Mar 24 12:24 templates.py
drwx------ 2 root root 4.0K Mar 23 13:32 trained_models/


In [23]:
# A dependency of the preprocessing for BERT inputs
!pip install  -q -U "tensorflow-text==2.8.*"

In [24]:
!pip install -q tf-models-official==2.7.0

In [25]:
from pathlib import Path
from datetime import datetime
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

import pandas as pd
import time


import time
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

from experiments import evaluate, save_results

print('Pandas v', pd.__version__)
#tf.get_logger().setLevel('ERROR')

Pandas v 1.4.4


In [26]:
 !nvidia-smi

Thu Mar 30 14:23:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |   1029MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
import configparser
from templates import DataManager, logger
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [29]:
#%%script echo skipping
results_dir = Path(config['results']['dir'])

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
file_name = f'results_{currentTime}.csv'
output_file = main_folder / results_dir / file_name



In [30]:
!python run_classical_MLs.py -o $file_name

03-30-2023 14:23:36 l:25| Loaded dataset with 30609 rows.
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
03-30-2023 14:23:37 l:121| Training model: xgboost
03-30-2023 14:23:46 l:129| Ended training xgboost in: 9.330539482999939s
No ensemble method selected in config.ini file.
header is True
03-30-2023 14:23:46 l:149| Results saved to results/results_230330_142335.csv


In [31]:
data_manager = DataManager(config)

In [32]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [33]:
# Manual selection
bert_model_name_list = ['bert_en_uncased_L-12_H-768_A-12', 
                        'bert_en_cased_L-12_H-768_A-12',
                        'small_bert/bert_en_uncased_L-2_H-128_A-2',
                        'small_bert/bert_en_uncased_L-4_H-512_A-8',
                        'small_bert/bert_en_uncased_L-8_H-128_A-2',
                        'small_bert/bert_en_uncased_L-12_H-768_A-12',
                        'bert_multi_cased_L-12_H-768_A-12',
                       'albert_en_base',
                        'electra_base',
                        'electra_small']

In [35]:
def build_classifier_model(local_tfhub_handle_preprocess, local_tfhub_handle_encoder):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Payload')
  preprocessing_layer = hub.KerasLayer(local_tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(local_tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [36]:
data_manager.x_train.shape[0]

24487

In [37]:
#main_dir = Path('/content/drive/MyDrive/Colab Notebooks/kasim/2023 Feb Bert SQLi/')
# 'SQLiV3_train.tsv', 'SQLiV3_test.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_testing.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
#train_file_name = 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
#train_file_name = 'SQLiV3_train.tsv'
#test_file_name = 'SQLiV3_test.tsv'
#train_file = Path(main_dir / train_file_name)
#test_file = Path(main_dir / test_file_name)

#print(f'Train file exists: {train_file.is_file()}')
#print(f'Test file exists: {test_file.is_file()}')


currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
#recording = Recorder(main_folder / results_dir / f'results_BERT_{currentTime}.csv')


AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
#seed = 42

x_train = data_manager.x_train
y_train = data_manager.y_train
x_test = data_manager.x_test
y_test = data_manager.y_test

# Create TensorFlow datasets for the training and validation sets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(8)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(8)

# Cache the training and validation datasets
train_dataset = train_dataset.cache()
val_dataset = val_dataset.cache()


# Prefetch the training and validation datasets
train_ds = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_ds = val_dataset.prefetch(buffer_size=AUTOTUNE)



#train_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = val_ds

class_names = ['payload','label']



In [38]:
tmp = train_ds.take(1)
for i in next(iter(tmp)):
  print(i)

tf.Tensor(
[b'if  (  6554  =  1534  )   select 6554 else drop function buqb--'
 b"admin'  )   or   (  '1'  =  '1'--"
 b' select * from users where id  =  1 or "&  )  " or 1  =  1 -- 1'
 b'callejon sur, 93 13-c' b"-6449'   )    )    or 2590  =  2848"
 b"1' in boolean mode  ) "
 b"1' and 5556  =    (  select count  (  *  )   from all_users t1,all_users t2,all_users t3,all_users t4,all_users t5  )  "
 b'SELECT MIN ( wood )  AS union FROM disease'], shape=(8,), dtype=string)
tf.Tensor([1 1 1 0 1 1 1 0], shape=(8,), dtype=int64)


In [39]:
for bert_model_name in bert_model_name_list:
  #bert_model_name = bert_model_name_list[0]
  print('****************************************************')
  #recording.set_current_method(f"{bert_model_name}" )
  tfhub_handle_encoder = map_name_to_handle[bert_model_name]
  tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

  print(f'BERT model selected           : {tfhub_handle_encoder}')
  print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


  bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
  bert_model = hub.KerasLayer(tfhub_handle_encoder)
  classifier_model = build_classifier_model(tfhub_handle_preprocess, tfhub_handle_encoder)

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  metrics = tf.metrics.BinaryAccuracy()

  epochs = 1 #modify this
  #steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
  steps_per_epoch = data_manager.x_train.shape[0]
  num_train_steps = (steps_per_epoch * epochs) #//200 
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')

  classifier_model.compile(optimizer=optimizer,
                          loss=loss,
                          metrics=metrics)


  print(f'Training model with {tfhub_handle_encoder}')
  start_time = time.time()
  history = classifier_model.fit(x=train_ds, #train_ds.take(2)
                                validation_data=test_ds, # validation_data=test_ds.take(2),
                                epochs=epochs)
  stop_time = time.time()
  training_time = (stop_time - start_time)*1000 / data_manager.notes['train_size']

  loss, accuracy = classifier_model.evaluate(test_ds)

  print(f'Loss: {loss}')
  print(f'Accuracy: {accuracy}')

  start_time = time.time()
  y_pred = classifier_model.predict(test_ds)
  stop_time = time.time()
  testing_time = (stop_time - start_time)*1000 / data_manager.notes['test_size']
  y_pred_np = tf.cast(tf.sigmoid(y_pred) > 0.5, tf.int32).numpy()
  X, y_true = zip(*test_ds.unbatch())
  y_true_np = [y.numpy() for y in y_true]
  #precision, recall, f1, support = precision_recall_fscore_support(y_true_np, y_pred_np, average='binary')
  #tn, fp, fn, tp = confusion_matrix(y_true_np, y_pred_np).ravel()

  notes = {'feature_method': bert_model_name,'model': bert_model_name, 
           'seed': data_manager.notes['seed'], 
           'split_ratio': data_manager.notes['split_ratio'],
           'train_size': data_manager.notes['train_size'],
           'test_size': data_manager.notes['test_size'],
           'extraction_time':0, 'feature_size':0,
           'train_time': training_time,
           'pred_time': testing_time, 'dataset': config['dataset']['file']}
  result = evaluate(y_true_np, y_pred_np, notes)
  print(result)
  save_results([result], output_file)


****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
2/2 [==============================] - 0s 16ms/step - loss: 0.7044 - binary_accuracy: 0.5625
Loss: 0.7043875455856323
Accuracy: 0.5625
{'accuracy': 0.500326690623979, 'precision': 0.10462776659959759, 'recall': 0.04575450945886494, 'f1_score': 0.06366697275788186, 'tp': 104, 'tn': 2959, 'fp': 890, 'fn': 2169, 'feature_method': 'small_bert/bert_en_uncased_L-2_H-128_A-2', 'model': 'small_bert/bert_en_uncased_L-2_H-128_A-2', 'seed': 42, 'split_ratio': 0.2, 'train_size': 24487, 'test_size': 6122, 'extraction_time': 0, 'feature_size': 0, 'train_time': 0.49966539393917386, 'pred_time': 1.9594392368350297, 'dataset': 'SQLiV3.tsv'}
Appending to the existing .cs